# Step-by-step through the experiment

*Want to follow along? {nb-download}`Download this notebook.<3. experiment step by step.ipynb>`*

## 1. Circuit generation

In [ ]:
from __future__ import annotations

import stim
from deltakit.circuit.gates import PauliBasis
from deltakit.explorer import Client, enums, types

client = Client.get_instance()
ion_gates = ["SQRT_XX", "X", "Y", "Z", "SQRT_X", "SQRT_X_DAG", "S", "S_DAG", "MZ", "RZ"]

experiment = types.QECExperimentDefinition(
    experiment_type=enums.QECExperimentType.QUANTUM_MEMORY,  # we perform a quantum memory experiment
    code_type=enums.QECECodeType.REPETITION,                 # we use repetition code for QEC
    observable_basis=PauliBasis.Z,                           # this is a Z-basis experiment. We reset and measure in Z
    num_rounds=1,                                            # smallest number of "waiting rounds"
    parameters=types.CircuitParameters.from_sizes([3]),      # this is distance-3 (can correct chains of errors up to 3)
    basis_gates=ion_gates,
)

# shortcut
experiment = types.QECExperimentDefinition.get_repetition_z_quantum_memory(
    3, 1, ion_gates)

circuit = client.generate_circuit(experiment)
display(stim.Circuit(circuit).diagram(type="timeline-svg"))

Here `M` gates represent measurements of physical qubits.
Two of these measurements are marked with `DETECTORS` annotation, meaning these measurement values will be used to construct a parity check (aka "syndrome", data for the decoder).

Next type of diagrams is useful to capture spatial and temporal relations:

In [ ]:
display(stim.Circuit(circuit).diagram(type="timeline-3d"))

You may also benefit from STIM and OpenQASM representations:

In [ ]:
print(circuit)
print(stim.Circuit(circuit).to_qasm(open_qasm_version=3, skip_dets_and_obs=False))

Experiment may me modified to capture you ideas. What if it ran for more error correction "rounds"?

In [ ]:
experiment.num_rounds = 3
circuit = client.generate_circuit(experiment_definition=experiment)
display(stim.Circuit(circuit).diagram(type="timeline-svg"))
# display(stim.Circuit(circuit).diagram(type="timeline-3d"))

## Exercise #3

Plot a 3D diagram for a rotated planar code, Z-memory experiment which has error distance 3 (sizes=[3, 3]), and 3 rounds of syndrome extraction (ancillary qubit measurement). Set basis gates to `None`.

In [ ]:
experiment = types.QECExperimentDefinition(
    ... # TODO
)
circuit = client.generate_circuit(experiment)
display(stim.Circuit(circuit).diagram(type="timeline-3d"))

**Takeways**:
- Experiment definition object in
- STIM circuit out

## 2. Noise

Realistic noise models are essential for two major purposes:
1. We supply decoders with valuable information about probabilities of different error mechanisms, and allow them to choose most likely reasons of observed detectors.
2. We enable realistic simulation of circuits, which is the only opportunity due to the lack of QPU hardware.

In [ ]:
experiment = types.QECExperimentDefinition.get_repetition_z_quantum_memory(3, 1, ion_gates)
circuit = client.generate_circuit(experiment_definition=experiment)
realistic_noise = types.PhysicalNoiseModel.get_ion_trap_noise()
# some realistic Ion Trap noise: https://arxiv.org/pdf/2406.12007
rqc_noise = types.PhysicalNoiseModel(
    t_1=53e-6,
    t_2=30e-6,
    time_1_qubit_gate=1e-6,
    time_2_qubit_gate=1e-6,
    time_measurement=32e-7,
    time_reset=200e-9,
    p_1_qubit_gate_error=1 - 0.9995,
    p_2_qubit_gate_error=1 - 0.963,
    p_reset_error=0.01,
    p_meas_qubit_error=1 - 0.9975,
    p_readout_flip=0.0,
)
noisy_circuit = client.add_noise(circuit, rqc_noise)
display(stim.Circuit(noisy_circuit).diagram(type="timeline-svg"))

**Takeways**:
- STIM circuit comes it
- "noisy" STIM circuit goes out

## 3. Simulation

Noisy circuit is a self-contained combination of experiment definition and a QPU. Unlike the qiskit/qasm execution paradigm, STIM allows noise to be the first class citizen of the quantum program. So, having a noisy circuit, you need nothing else to simulate.

This is how a noiseless circuit will execute:

In [ ]:
measurements, leakage = client.simulate_stim_circuit(circuit, shots=30)  # we don't use leakage for now

detectors, observables = measurements.to_detectors_and_observables(circuit)

And these are results for a noisy circuit:

In [ ]:
noisy_measurements, leakage = client.simulate_stim_circuit(noisy_circuit, shots=30)

detectors, observables = noisy_measurements.to_detectors_and_observables(noisy_circuit)

**Takeways**:
- Noisy STIM comes in
- a 0/1 table (`numpy` array) goes out. Rows represent "shots". In measurements table columns are measurements in the order of their appearance. (NB: we display them transposed for a compact view)

## Decoding

In [ ]:
decoder = types.Decoder(decoder_type=enums.DecoderType.LCD)

decoding_result = client.decode(
    detectors=detectors,
    observables=observables,
    decoder=decoder,
    noisy_stim_circuit=noisy_circuit
)

def print_result(decoder: str, result: types.DecodingResult):
    pass


print_result("LCD", decoding_result)

**Takeways**:
- Circuit, detectors and observables (0/1 tables) come in
- Predictions table and statistics goes out